In [1]:
import pandas as pd

# Charger le premier fichier restaurants
df1 = pd.read_csv('data/restaurant_1_week_002.csv')

# Charger le deuxième fichier restaurants
df2 = pd.read_csv('data/restaurant_1_week_007.csv')

# Afficher les premières lignes du premier fichier
print("Premières lignes du premier fichier :")
print(df1.head())

# Afficher les premières lignes du deuxième fichier
print("Premières lignes du deuxième fichier :")
print(df2.head())

Premières lignes du premier fichier :
   Order Number           Order Date      Item Name  Quantity  Product Price  \
0          1388  2015-01-10 20:12:00  Onion Chutney         1           0.50   
1          1388  2015-01-10 20:12:00     Mint Sauce         1           0.50   
2          1388  2015-01-10 20:12:00    Lime Pickle         1           0.50   
3          1388  2015-01-10 20:12:00        Paratha         1           2.95   
4          1387  2015-01-10 16:55:00    Onion Bhaji         1           3.95   

   Total products  
0               6  
1               6  
2               6  
3               6  
4               1  
Premières lignes du deuxième fichier :
   Order Number           Order Date      Item Name  Quantity  Product Price  \
0          1397  2015-02-10 14:38:00  Spicy Papadum         1           0.80   
1          1403  2015-02-10 17:29:00        Paratha         1           2.95   
2          1403  2015-02-10 17:29:00   Coke 1.5 ltr         1           2.95   
3 

In [2]:
# Afficher les statistiques descriptives du premier fichier
print("\nStatistiques descriptives du premier fichier :")
print(df1.describe())

# Afficher les statistiques descriptives du deuxième fichier
print("\nStatistiques descriptives du deuxième fichier :")
print(df2.describe())

# Afficher les types de données du premier fichier
print("\nTypes de données du premier fichier :")
print(df1.dtypes)

# Afficher les types de données du deuxième fichier
print("\nTypes de données du deuxième fichier :")
print(df2.dtypes)

# Vérifier les valeurs manquantes dans le premier fichier
print("\nValeurs manquantes dans le premier fichier :")
print(df1.isnull().sum())

# Vérifier les valeurs manquantes dans le deuxième fichier
print("\nValeurs manquantes dans le deuxième fichier :")
print(df2.isnull().sum())


Statistiques descriptives du premier fichier :
       Order Number   Quantity  Product Price  Total products
count     21.000000  21.000000      21.000000       21.000000
mean    1346.761905   1.047619       4.933333        4.428571
std      164.304871   0.218218       2.611529        2.226464
min      630.000000   1.000000       0.500000        1.000000
25%     1376.000000   1.000000       3.950000        3.000000
50%     1385.000000   1.000000       4.950000        3.000000
75%     1386.000000   1.000000       5.950000        7.000000
max     1388.000000   2.000000       9.950000        7.000000

Statistiques descriptives du deuxième fichier :
       Order Number   Quantity  Product Price  Total products
count     19.000000  19.000000      19.000000       19.000000
mean    1398.736842   1.052632       5.994737        6.368421
std        5.300337   0.229416       4.072158        2.265944
min     1389.000000   1.000000       0.800000        2.000000
25%     1397.000000   1.000000     